In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
import tensorflow as tf
from pathlib import Path
from keras.preprocessing import image
import numpy as np



**Image Classification**

In [0]:
#load dataset
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
#normalizing data
x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255

In [0]:
#converting labels from 0-9 in an array having one element 1 at a time corresponding to class
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

In [0]:
#create a model
model = Sequential()
model.add(Conv2D(32,(3,3),padding='same',activation='relu',input_shape=(32,32,3)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)         

In [0]:
#compile the nn
model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)         

In [0]:
#train the nn
model.fit(x_train,y_train,batch_size=64,epochs=30,validation_data=(x_test,y_test),shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 6s 122us/step - loss: 1.0453 - accuracy: 0.6298 - val_loss: 0.9254 - val_accuracy: 0.6761
Epoch 2/30
50000/50000 [==============================] - 6s 121us/step - loss: 0.9201 - accuracy: 0.6761 - val_loss: 0.7962 - val_accuracy: 0.7264
Epoch 3/30
50000/50000 [==============================] - 6s 120us/step - loss: 0.8415 - accuracy: 0.7034 - val_loss: 0.7539 - val_accuracy: 0.7438
Epoch 4/30
50000/50000 [==============================] - 6s 121us/step - loss: 0.7837 - accuracy: 0.7246 - val_loss: 0.7348 - val_accuracy: 0.7455
Epoch 5/30
50000/50000 [==============================] - 6s 122us/step - loss: 0.7359 - accuracy: 0.7407 - val_loss: 0.7023 - val_accuracy: 0.7565
Epoch 6/30
50000/50000 [==============================] - 6s 120us/step - loss: 0.6946 - accuracy: 0.7560 - val_loss: 0.6758 - val_accuracy: 0.7660
Epoch 7/30
50000/50000 [==============================] - 6s 1

In [0]:
#save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)


4555

In [0]:
#save model weights
model.save_weights("model.h5")


In [0]:
#prediction
class_labels = ["Plane","Car","Bird","Cat","Deer","Dog","Frog","Horse","Boat","Truck"]

In [0]:
#load json file that contains model structure
f = Path("model_structure.json")
model_structure = f.read_text()

In [0]:
#recreate model structure
model = tf.keras.models.model_from_json(model_structure)

In [0]:
#reload models weights
model.load_weights("model.h5")

In [0]:
#load test image
img = image.load_img("/content/cat.png",target_size = (32,32))

In [0]:
#converting to array
img_test = image.img_to_array(img)/255
img_list = np.expand_dims(img_test,axis=0)

In [0]:
#predicting
results = model.predict(img_list)
#since testing only one img we need only first index
sing_rslt  = results[0]
#getting likelyhood score
most_like_index = int(np.argmax(sing_rslt))
class_likelihood = sing_rslt[most_like_index]
#getting label
class_label = class_labels[most_like_index] 
print("this image is a {} and class likelihood : {:.2f}".format(class_label,class_likelihood))

this image is a Cat and class likelihood : 0.83


**Object detection**

In [0]:
from keras.applications import vgg16

In [0]:
model1 = vgg16.VGG16()

553467904/553467096 [==============================] - 16s 0us/step


In [0]:
img1 = image.load_img("bay.jpg",target_size=(224,224))
x = image.img_to_array(img1)
x= np.expand_dims(x,axis=0)

In [0]:
#preprocess
x = vgg16.preprocess_input(x)

In [0]:
#prediction
predictions = model1.predict(x)
predicted_classes = vgg16.decode_predictions(predictions,top =9)
print("Top predictions for this image are:")

for imagenet_id,name,likelihood in predicted_classes[0]:
  print("Prediction : {}-{:.2f} ".format(name,likelihood))


Top predictions for this image are:
Prediction : seashore-0.40 
Prediction : promontory-0.33 
Prediction : lakeside-0.12 
Prediction : breakwater-0.06 
Prediction : sandbar-0.05 
Prediction : cliff-0.01 
Prediction : dock-0.01 
Prediction : boathouse-0.00 
Prediction : valley-0.00 
